<a href="https://colab.research.google.com/github/1975JHK/1975JHK.github.io/blob/main/Performance_with_Bagging_and_OOD_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
## 4. OOD Detection --------------------------------------------------------
new.result <- data.frame(trail.1 = rep(NA, times = nrow(test)))
for(i in 1:101){
  trial <- c(rep(NA, times = nrow(test)))
  new.result <- data.frame(new.result, trial)}

new.result2 <- cbind(
  new.result[ , -c(1:2)],
  majority = rep(NA, times = nrow(test)),
  count = rep(NA, times = nrow(test)),
  rate = rep(NA, times = nrow(test))
)


for(i in 1:100){
  index <- sample(1:nrow(train), nrow(train)*0.8, replace = F)
  new.train <- train[index, ]
  new.svm <- svm(Y3 ~ ., data = new.train[ , c(1:26)],
                 type = type,
                 kernel = best2[1, 1],
                 cost = best2[1, 2],
                 gamma = best2[1, 3],
                 epsilon = best2[1, 4])
  new.pred = predict(new.svm, newdata = test[ , c(1:25)])
  new.result2[ , i] <- new.pred
}

new.result2$majority <- apply(new.result2[ , 1:100], 1, function(x) names(which.max(table(x))))
new.result2$count <- apply(new.result2[ , 1:100], 1, function(x) max(table(x)))
new.result2$rate <- new.result2$count/100

finals_sh <- data.frame(new.result2[ , c(101:103)], test$Y3)
test_kit2 <- data.frame(rate = seq(0.800, 0.999, by = 0.001),
                        accuraty = rep(NA, length = 200))
for(i in 1:nrow(test_kit2)){
  finals_sh2 <- finals_sh %>% filter(test_kit2[i, 1] < rate)
  cm = confusionMatrix(as.factor(finals_sh2$majority), 
                       finals_sh2$test.Y3)
  test_kit2[i, 2] = cm$overall[1]
}

best_rate2 <- test_kit2[which.max(test_kit2[ , 2]), 1]
finals_sh <- finals_sh %>% filter(best_rate2[1] <= rate)

cm4 = confusionMatrix(as.factor(finals_sh$majority), finals_sh$test.Y3)
print(cm4)

concolusion4 <- rep(NA, times = 3)
concolusion4[1] = cm4$table[4]/(cm4$table[3] + cm4$table[4])
concolusion4[2] = cm4$table[1]/(cm4$table[1] + cm4$table[2])
concolusion4[3] = (cm4$table[1] + cm4$table[4])/
  (cm4$table[1] + cm4$table[2] + cm4$table[3] + cm4$table[4])

Confusion Matrix and Statistics

          Reference
Prediction Good Bad
      Good   72   2
      Bad     3  50
                                          
               Accuracy : 0.9606          
                 95% CI : (0.9105, 0.9871)
    No Information Rate : 0.5906          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.9188          
                                          
 Mcnemar's Test P-Value : 1               
                                          
            Sensitivity : 0.9600          
            Specificity : 0.9615          
         Pos Pred Value : 0.9730          
         Neg Pred Value : 0.9434          
             Prevalence : 0.5906          
         Detection Rate : 0.5669          
   Detection Prevalence : 0.5827          
      Balanced Accuracy : 0.9608          
                                          
       'Positive' Class : Good            
                           